## Tasks:
### Eda
The main tasks to do in Exploratory Data Analysis:
* Null vars by column
* Evaluate id, for duplicated cases.
* Describe our target var, quantils, standard deviation(S),avg,min,max,distribution, bucketized.
* Describe categorical features: each category how is related to avg metric(*some categorical features could be changed to numerical ones -i.e. Excelent, medium,bad - in order to avoid make binary poolQuality*).
* Describe numerical features: distribution(*relation with target? in same graph*), covariance with targeted var.     

### Modeling
You should complete this !!!

In [ ]:
from edautils import check_duplicates,fig_axes_subplots,set_dark
set_dark()
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", context="notebook")
#plt.style.use("dark_background")

train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
train.head(3)

In [ ]:
pd.set_option("display.max_rows", 90)
train.head(3).T

In [ ]:
TARGET = 'SalePrice'
ID = 'Id'

## Check Null values

In [ ]:
import matplotlib.pyplot as plt
nulls = train.isnull().sum().sort_values(ascending=False)
nulls = nulls[nulls > 0]
g = sns.barplot(x=nulls.index,y=nulls.values)
g.set_xticklabels(g.get_xticklabels(), rotation=90)
plt.show()

In order to train a model es mandatory fix this null values. We have some columns with over than 90% of null values!, in some cases is a reference to a new feature, i.e., PoolQC is the Pool Quality if you don't have a Pool you have a null value in this. So the new feature can be, Has a Pool?.

In [ ]:
check_duplicates(train[ID])

## Evaluating Target data
We need to understand our target data, because all the project are related to this. What we look for? specifically nothing, but we need understand very well this, because when we analyse some other features, we will want to correlate with this.

In [ ]:
train[TARGET].describe()

In [ ]:
train[TARGET].hist(bins=50)

In [ ]:
 g = sns.boxplot(x=train[TARGET])

### Evaluating Categorical columns:
We have ~90 features columns, we want to understand how much are categorical, and if this "categories" are related in some way with our target var. Additionally we want to think about the best representation in our model.

In [ ]:
categorical_cols = list(train.select_dtypes(include='object').columns)
print(categorical_cols)

In [ ]:
def describe_categorical(dataframe,category):
    d = dataframe.groupby(category).agg({ID:['count'],TARGET:['mean','std']})
    d.columns = d.columns.get_level_values(1).astype(str)
    d['share'] = round(100*d['count']/d['count'].sum(),2)
    #d['share'] = d['share'].map('{:.2f}'.format)
    d['mean'] = d['mean'].map('{:,.2f}'.format)
    d['std'] = d['std'].map('{:,.2f}'.format)
    
    return d

In [ ]:
train[categorical_cols] = train[categorical_cols].fillna("--NAN--")
for c in categorical_cols:
    print(describe_categorical(train,c))

In [ ]:
def describe_categorical_graph(df, cols, fig_size=(18,20)):
    # Used in next graphs to print the same order inner category options
    order = {}
    
    fig, axs = plt.subplots(len(cols), 2, sharey=False, figsize=fig_size)
    
    def boxplot(df,col,target,ax,order=None):
        g = sns.boxplot(x=col, y=target, ax=axs[ic,0], data=df, order=order)#, order=order[c])
        g.set_xlabel("")
        return g
    
    def barplot(df,col,ax):
        ys = list(df.share.values)
        xs = list(df.index.values)
        g = sns.barplot(x=xs, y=ys, ax=ax)
    
        for ix,y in enumerate(ys):
            col_text = f'{df["count"].values[ix]}\n{round(ys[ix],1)}%'
            g.text(ix, y+0.01, col_text, color='black', ha="center", fontsize=10)
        return g
    
    for ic,c in enumerate(cols):
        d = describe_categorical(df,c).sort_values(by='share',ascending=False)
        g0 = boxplot(df,c,TARGET,axs[ic,0], order=d.index.values)
        g0.set_xticklabels(g0.get_xticklabels(), rotation=90, horizontalalignment='right')
        g0.set_title(c)
        g1 = barplot(d, c, axs[ic,1])
        g1.set_xticklabels(g1.get_xticklabels(), rotation=90, horizontalalignment='right')
        
    fig.tight_layout()

In [ ]:
describe_categorical_graph(train, categorical_cols, (16,150))

Interesting things about the graphs:
* Looking our statistics(Quartiles,Avg,etc) for each categorical column, we could see the category impact in our target var(Sale Price). And in general all our variables are related in some way with that.
* But seeing the category share in each column, in specific categories,  all data are concentrated in just one value. One approach is just use categories with less than 95% of share in one value(over than ~70 cases in other values).

### Evaluating Numerical columns:


In [ ]:
numerical_cols = [c for c in train.columns if c not in categorical_cols]
numerical_cols.remove(ID)
numerical_cols.remove(TARGET)
print(numerical_cols)

In [ ]:
for c in numerical_cols:
    train.plot(kind='scatter', x=c, y=TARGET)#, s=df.col3)

In [ ]:
import seaborn as sns
corr_matrix = train[numerical_cols+[TARGET]].corr().abs()
sns.heatmap(corr_matrix, vmin=0, vmax=1)#.astype(int)

In [ ]:
#corm.SalePrice[np.logical_or(corm.SalePrice > 0.2, corm.SalePrice < -0.2)]
corr_matrix.SalePrice[corr_matrix.SalePrice.abs() < 0.2].abs().sort_values()

## Modeling

### Encoding Categorical variables
We will encode categorical variables with ordinal values.

In [ ]:
def linear_mean_encoder(frame, feature):
    ordering = frame[[feature,TARGET]].groupby(feature).mean()
    ordering = ordering.sort_values(by=TARGET, ascending=True)
    ordering = ordering.reset_index()
    ordering.index = ordering.index+1
    feature_cateogory = ordering[feature].to_dict()

    for order,category in feature_cateogory.items():
        frame.loc[frame[feature] == category, 'E_'+feature] = order

for col in categorical_cols:
    linear_mean_encoder(train,col)

### Solving Null values in numerical columns
We will solve null values looking for most correlated column, and using it in order to regression missing values.

In [ ]:
numerical_null = train[numerical_cols].isnull().sum()
numerical_null = numerical_null[numerical_null>0].sort_values(ascending=False)
numerical_null

In [ ]:
from sklearn import linear_model
def columns_best_descriptors(data, columns_to_describe, avoided_columns =[]):
    numeric_null_correlations = data.corr()[columns_to_describe].abs()
    descriptors = {}
    for c in numeric_null_correlations.columns:
        descriptors_category =  numeric_null_correlations[c].sort_values(ascending=False)
        avoided_columns = list(numerical_null.index)+[c]
        descriptors[c]=str(descriptors_category[~descriptors_category.index.isin(avoided_columns)].head(1).index.values[0])
    return descriptors

descriptors = columns_best_descriptors(data = train, 
                                       columns_to_describe = list(numerical_null.index.values), 
                                       avoided_columns = list(numerical_null.index.values))

print(descriptors)
def simple_linear_models_with_dict(train, y_x_dict_columns):
    models = {}
    for y_column, x_column in y_x_dict_columns.items():
        d = train[~train[y_column].isnull()]
        y = d[y_column]
        x = d[x_column].to_numpy().reshape(-1,1)
        models[y_column] = linear_model.LinearRegression()
        models[y_column].fit(x, y)
    return models

models = simple_linear_models_with_dict(train, descriptors)

for y_column,x_column in descriptors.items():
    synthetic_col = "S_"+y_column
    train[synthetic_col] = train[y_column]
    for ix, r in train[train[synthetic_col].isnull()].iterrows():
        syntetic_value = models[y_column].predict([[r[x_column]]])
        train.at[ix,synthetic_col] = syntetic_value
    print(f"Synthetic col: {synthetic_col} was created successfully for {y_column}")

#### Checking syntetic values
We will make some basics checks, i.e, check some statistics about the distribution

In [ ]:
train['S_LotFrontage'].isnull().sum()

In [ ]:
train[['LotFrontage','S_LotFrontage']].describe()

### Training models

Removing columns with nulls (we have synthetized ones), and categorical ones(we have encoded ones)

In [ ]:
X = train.copy()
Y = train[TARGET].copy()
X.drop(labels=[ID,TARGET], axis=1, inplace=True)
X.drop(labels=list(descriptors.keys()), axis=1, inplace=True)
X.drop(labels=categorical_cols, axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=10) 
print(f"train size : {len(y_train)}")
print(f"test size : {len(y_test)}")

In [ ]:
print(list(x_train.columns))

## Selecting our model
We want to select the best model to predict our price, but depending about our organization, we will select our cost function. I.e, if our company loss money if we have a bad prediction in large amounts, maybe we will use a mean_squared_error to select the best one. But if we need to find the best model representation without focus in large amount "squared r" is a better option.

In [ ]:
from sklearn import linear_model
parameters = {'normalize':[True,False],'fit_intercept':[True,False]}
model = linear_model.Ridge()
reg = GridSearchCV(model, parameters, cv=5, scoring='r2')
reg.fit(x_train, y_train)

In [ ]:
reg.best_score_

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score

preds = reg.best_estimator_.predict(x_test)

print('Mean squared error: %.2f' % mean_squared_error(y_test, preds))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.4f' % r2_score(y_test, preds))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(random_state = 0)
reg.fit(x_train, y_train)

In [ ]:
preds = reg.predict(x_test)
print('Mean squared error: %.2f' % mean_squared_error(y_test, preds))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.4f' % r2_score(y_test, preds))